# FMM Recbole 구현

### Recbole 라이브러리 로딩

In [2]:
# !pip install recbole

In [44]:
# !pip install ray

     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.4 MB ? eta -:--:--━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/57.4 MB 118.4 MB/s eta 0:00:01━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/57.4 MB 61.8 MB/s eta 0:00:01━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/57.4 MB 93.1 MB/s eta 0:00:01━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/57.4 MB 160.3 MB/s eta 0:00:01━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/57.4 MB 174.8 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━ 27.7/57.4 MB 177.9 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━ 33.7/57.4 MB 172.7 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━ 39.8/57.4 MB 174.2 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━ 45.9/57.4 MB 173.1 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━ 52.1/57.4 MB 178.2 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 57.4/57.4 MB 181.0 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 57.4/57.4 MB 181.0 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━

In [58]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

from logging import getLogger
import os
import json
import numpy as np
import pandas as pd
import time, datetime
from tqdm import tqdm

from recbole.model.context_aware_recommender.ffm import FFM

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.data.interaction import Interaction
from recbole.utils import init_logger, get_trainer, init_seed, set_color, get_model
from recbole.quick_start.quick_start import load_data_and_model

from recbole.config import Config
from recbole.data import create_dataset

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import ndcg_score, recall_score

import torch

### 데이터 로드

In [59]:
train_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv')

In [60]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5154471 entries, 0 to 5154470
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   user    int64
 1   item    int64
 2   time    int64
dtypes: int64(3)
memory usage: 118.0 MB


In [63]:
data_path = '/opt/ml/input/data/train'
year_data = pd.read_csv(os.path.join(data_path, 'years.tsv'), sep='\t')
writer_data = pd.read_csv(os.path.join(data_path, 'writers.tsv'), sep='\t')
title_data = pd.read_csv(os.path.join(data_path, 'titles.tsv'), sep='\t')
genre_data = pd.read_csv(os.path.join(data_path, 'genres.tsv'), sep='\t')
director_data = pd.read_csv(os.path.join(data_path, 'directors.tsv'), sep='\t')

In [64]:
df_merge = pd.merge(train_df, year_data.drop_duplicates(subset=['item']), on='item', how='inner')
df_merge = pd.merge(df_merge, writer_data.drop_duplicates(subset=['item']), on='item', how='inner')
df_merge = pd.merge(df_merge, title_data.drop_duplicates(subset=['item']), on='item', how='inner')
df_merge = pd.merge(df_merge, genre_data.drop_duplicates(subset=['item']), on='item', how='inner')
df_merge = pd.merge(df_merge, director_data.drop_duplicates(subset=['item']), on='item', how='inner')

In [65]:
train_data = df_merge[['user', 'item', 'time']]

In [66]:
user_data = df_merge[['user']]

In [67]:
item_data = df_merge[['item', 'year', 'writer', 'title', 'genre', 'director']].drop_duplicates(subset=['item'])

In [68]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4658299 entries, 0 to 4658298
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   user    int64
 1   item    int64
 2   time    int64
dtypes: int64(3)
memory usage: 142.2 MB


In [69]:
userid, itemid = list(set(train_data.user)), list(set(train_data.item))
n_user, n_item = len(userid), len(itemid)
print(f'n_user : {n_user}')
print(f'n_item : {n_item}')

n_user : 31360
n_item : 4967


### 데이터 전처리

### 데이터 파일 변환

기존 데이터 파일을 Recbole 데이터 파일로 변환시키는 과정

In [70]:
userid, itemid = sorted(userid), sorted(itemid)
n_user, n_item = len(userid), len(itemid)

userid_2_index = {v:i for i,v in enumerate(userid)}
itemid_2_index = {v:i for i,v in enumerate(itemid)}
index_2_userid = {i:v for i,v in enumerate(userid)}
index_2_itemid = {i:v for i,v in enumerate(itemid)}

In [71]:
yamldata = """
field_separator: "\t"
USER_ID_FIELD: user_id
ITEM_ID_FIELD: item_id
TIME_FIELD: timestamp

load_col:
    inter: [user_id, item_id, timestamp]
    user: [user_id]
    item: [item_id, year, writer, title, genre, director]

train_neg_sample_args:
    uniform: 1
    
eval_args:
    split: {'RS': [4, 1, 1]}
    group_by: user
    order: RO
    mode: full
metrics: ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision', 'MAP']
topk: 10
valid_metric: Recall@10
"""

In [72]:
train_data.user = train_data.user.map(userid_2_index)
train_data.item = train_data.item.map(itemid_2_index)

user_data.user = user_data.user.map(userid_2_index)
item_data.item = item_data.item.map(itemid_2_index)

/tmp/ipykernel_31790/1297655130.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.user = train_data.user.map(userid_2_index)
/tmp/ipykernel_31790/1297655130.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.item = train_data.item.map(itemid_2_index)
/tmp/ipykernel_31790/1297655130.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [73]:
train_data.columns=['user_id:token', 'item_id:token', 'timestamp:float']
user_data.columns=['user_id:token']
item_data.columns=['item_id:token', 'year:float', 'writer:token', 'title:token', 'genre:token', 'director:token']


In [74]:
outpath = f"dataset/train_data"
# outfile = f"dataset/train_data/train_data.inter"
yamlfile = f"train_data.yaml"

os.makedirs(outpath, exist_ok=True)
# sub_train=train.groupby("user").sample(n=10, random_state=SEED)
# sub_train.shape

# print("Processing Start")
# inter_table = []
# for user, item, time in zip(train_data.user, train_data.item, train_data.time):
#     uid, iid = userid_2_index[user], itemid_2_index[item]
#     # tval = int(time.mktime(datetime.datetime.strptime(time, "%Y-%m-%d %H:%M:%S").timetuple()))
#     inter_table.append( [uid, iid, time] )

# print("Processing Complete")

print("Dump Start")
# 데이터 설정 파일 저장
with open(yamlfile, "w") as f:
    f.write(yamldata) 

# 데이터 파일 저장
train_data.to_csv(os.path.join(outpath,"train_data.inter"),sep='\t',index=False)
user_data.to_csv(os.path.join(outpath,"train_data.user"),sep='\t',index=False)
item_data.to_csv(os.path.join(outpath,"train_data.item"),sep='\t',index=False)
# with open(outfile, "w") as f:
#     # write header
#     f.write("user_id:token\titem_id:token\ttimestamp:float\n")
#     for row in inter_table:
#         f.write("\t".join([str(x) for x in row])+"\n")

print("Dump Complete")

Dump Start


447

Dump Complete


### 로거 생성

In [75]:
logger = getLogger()

### 설정 인스턴스 생성

In [76]:
# configurations initialization
config = Config(model='FFM', dataset="train_data", config_file_list=[f'train_data.yaml'])
config['epochs'] = 1
config['show_progress'] = False
config['device'] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)

logger.info(config)

22 Dec 10:40    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/train_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 1
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'uniform': 1, 'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [4, 1, 1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision', 'MAP']
topk = [10]
valid_metric = Recall@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_se

### 데이터 로드

In [78]:
# dataset filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

22 Dec 10:41    INFO  train_data
The number of users: 31361
Average actions of users: 148.54269770408163
The number of items: 4968
Average actions of items: 937.8496074088987
The number of inters: 4658299
The sparsity of the dataset: 97.01010545165151%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'year', 'writer', 'title', 'genre', 'director']
22 Dec 10:41    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'uniform': 1, 'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
22 Dec 10:41    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [4, 1, 1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}]


In [79]:
train_data.dataset
valid_data.dataset
test_data.dataset

train_data
The number of users: 31361
Average actions of users: 99.86195790816326
The number of items: 4968
Average actions of items: 630.4954701026777
The number of inters: 3131671
The sparsity of the dataset: 97.98996027302648%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'year', 'writer', 'title', 'genre', 'director', 'label']

train_data
The number of users: 31361
Average actions of users: 24.340369897959185
The number of items: 4968
Average actions of items: 153.67706865311052
The number of inters: 763314
The sparsity of the dataset: 99.51007258931251%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'year', 'writer', 'title', 'genre', 'director', 'label']

train_data
The number of users: 31361
Average actions of users: 24.340369897959185
The number of items: 4968
Average actions of items: 153.67706865311052
The number of inters: 763314
The sparsity of the dataset: 99.51007258931251%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'year', 'writer', 'title', 'genre', 'director', 'label']

### 모델 인스턴스 생성

In [80]:
# model loading and initialization
init_seed(config['seed'], config['reproducibility'])
model = FFM(config, train_data.dataset).to(config['device'])
logger.info(model)

22 Dec 10:41    INFO  FFM(
  (token_embedding_table): FMEmbedding(
    (embedding): Embedding(44399, 10)
  )
  (first_order_linear): FMFirstOrderLinear(
    (token_embedding_table): FMEmbedding(
      (embedding): Embedding(44399, 1)
    )
  )
  (sigmoid): Sigmoid()
  (ffm): FieldAwareFactorizationMachine(
    (token_embeddings): ModuleList(
      (0): Embedding(44399, 10)
      (1): Embedding(44399, 10)
      (2): Embedding(44399, 10)
      (3): Embedding(44399, 10)
      (4): Embedding(44399, 10)
      (5): Embedding(44399, 10)
    )
  )
  (loss): BCEWithLogitsLoss()
)
Trainable parameters: 3152330


### 모델 학습

In [81]:
# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(
    train_data, valid_data, saved=True, show_progress=config['show_progress']
)

22 Dec 10:42    INFO  epoch 0 training [time: 34.38s, train loss: 1302.9981]
22 Dec 10:43    INFO  epoch 0 evaluating [time: 99.88s, valid_score: 0.086800]
22 Dec 10:43    INFO  valid result: 
recall@10 : 0.0868    mrr@10 : 0.382    ndcg@10 : 0.1879    hit@10 : 0.7531    precision@10 : 0.1746    map@10 : 0.0898
22 Dec 10:43    INFO  Saving current: saved/FFM-Dec-22-2022_10-41-38.pth


### 학습 결과 출력

In [82]:
# model evaluation
test_result = trainer.evaluate(test_data, load_best_model="True", show_progress=config['show_progress'])

logger.info(set_color('best valid ', 'yellow') + f': {best_valid_result}')
logger.info(set_color('test result', 'yellow') + f': {test_result}')

result = {
    'best_valid_score': best_valid_score,
    'valid_score_bigger': config['valid_metric_bigger'],
    'best_valid_result': best_valid_result,
    'test_result': test_result
}

print(json.dumps(result, indent=4))

22 Dec 10:43    INFO  Loading model structure and parameters from saved/FFM-Dec-22-2022_10-41-38.pth
22 Dec 10:45    INFO  best valid : OrderedDict([('recall@10', 0.0868), ('mrr@10', 0.382), ('ndcg@10', 0.1879), ('hit@10', 0.7531), ('precision@10', 0.1746), ('map@10', 0.0898)])
22 Dec 10:45    INFO  test result: OrderedDict([('recall@10', 0.1023), ('mrr@10', 0.4591), ('ndcg@10', 0.2405), ('hit@10', 0.7815), ('precision@10', 0.2181), ('map@10', 0.1329)])


{
    "best_valid_score": 0.0868,
    "valid_score_bigger": true,
    "best_valid_result": {
        "recall@10": 0.0868,
        "mrr@10": 0.382,
        "ndcg@10": 0.1879,
        "hit@10": 0.7531,
        "precision@10": 0.1746,
        "map@10": 0.0898
    },
    "test_result": {
        "recall@10": 0.1023,
        "mrr@10": 0.4591,
        "ndcg@10": 0.2405,
        "hit@10": 0.7815,
        "precision@10": 0.2181,
        "map@10": 0.1329
    }
}


### Inference

In [31]:
model_path='saved/FFM-Dec-22-2022_08-26-29.pth'